In [1]:
import pickle
a = open("Articles.pickle",'rb')
uncleanedarticles = pickle.load(a)
articles=[]
text=""
title=""
titles=[]
sources=[]
for i in uncleanedarticles:
#     print(i)
    source,url,title,text = i[0],i[1],i[2],i[3]
    sources.append(source)
    text = text.replace('\n\n','\n')
#     print(text)
    if text =="":
        continue
#     article = i.split('\n\n\n\n\n')
#     text = article[1].strip('\n').strip(' ')
    if text.count('\n') <10:
        continue
    if text.count('https://') >10:
        continue   
#     title = article[0].strip('\n').strip(' ')
    title = title.strip(' ')
    if title in titles:
        continue
    titles.append(title)
    articles.append([source,url,title,text])
a.close()
# print(articles)

In [2]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ps = SnowballStemmer('english')
wordRepArticles =[]
idf={}

articleslen=[]
for i in articles:
    title,text = i[2],i[3]
    text = text.replace('\\','');text = text.replace('.\n','\n');text = text.replace('\n\n','\n')
    articledist={}
    sentencelis=[]
    count=0
    for i in (text+'\n'+title).split('\n'):
        if i ==title:
            mult =2
        else:
            mult =1
        linedist={}
        v = i.split(' ')
        if len(v)<4:
            continue
        for word in v:
            if word not in stopwords.words('english'):
                word = word.replace('(','');word = word.replace(')','')
                word = word.replace('"','');word = word.replace(',','');word = word.strip(' ')
                stemmed = ps.stem(word)
                if stemmed in articledist.keys():
                    articledist[stemmed]+=mult
                else:
                    articledist[stemmed]=mult
                if stemmed in linedist.keys():
                    linedist[stemmed]+=1
                else:
                    linedist[stemmed]=1
            count+=1
        sentencelis.append([i,linedist])
    for words in articledist.keys():   
        if words in idf.keys():
            idf[words]+=1
        else:
            idf[words]=1
    wordRepArticles.append([title,articledist,sentencelis])
    articleslen.append(count)

In [3]:
print(len(wordRepArticles))
# wordRepArticles[0]
print(len(articles))
print(len(uncleanedarticles))
# fileObject = open('cleanedarticles.pickle','wb')
# pickle.dump(articles,fileObject)
# fileObject.close()

2151
2151
5314


In [4]:
articles[:5]

[['abcnews',
  'https://abcnews.go.com/US/reads-monday/story?id=59712958',
  'Must reads before Monday',
  'One parent\'s decision to shame his daughter on social media drew mixed responses, Florida is struggling with major toxic algae blooms, the only operating spaceport in the world is crumbling, and eight children affected by Trump\'s "zero-tolerance" border policy are still waiting to be reunited with their families.\nDon’t miss out on some of ABC News\' best reporting from this week.\n\'What are you trying to prove?\': Parenting experts weigh in on dad\'s bullying punishment\nSTOCK/Getty Images\nA viral video posted by a father that forced his daughter to walk 5 miles to school in near-freezing temperatures has prompted questions about the use of social media as a way to humiliate and discipline children. Some experts say that shame is not an "effective way to change behavior."\n\'When is this going to stop?\': Florida activist highlights devastating impact of algae blooms on wild

In [5]:
import math
n = len(articles)
avgArticleLen = sum(articleslen)*1.0/n
match = [[0 for i in range(n)] for i in range(n)]
for i in range(n):
    mainarticle = wordRepArticles[i]
    mainlength = articleslen[i]
    for j in range(i+1,n):
        comparearticle = wordRepArticles[j]
        comparelength = articleslen[j]
        commonwords = list(set(mainarticle[1].keys()) & set(comparearticle[1].keys())) 
        value = 0
        for word in commonwords:
            mainvalue = (1+math.log(mainarticle[1][word]))*1.0/(math.log(1+mainlength/avgArticleLen))
            compvalue = (1+math.log(comparearticle[1][word]))*1.0/(math.log(1+comparelength/avgArticleLen))
            value += mainvalue*compvalue* math.log(1+n/idf[word])
        match[i][j] = value
        match[j][i] = value

In [6]:
topmatches = [[] for i in range(n)]
matches = 3
for i in range(n):
    vals = [[match[i][j],j] for j in range(n)]
    toparticle = sorted(vals,reverse=True)[:matches]
    print("Main title: "+ articles[i][0])
    for j in range(matches):
        topmatches[i].append(toparticle[j])
        print(str(j)+"/" + str(matches)+ ": " + articles[toparticle[j][1]][0])

Main title: abcnews
0/3: usatoday
1/3: aol
2/3: abcnews
Main title: abcnews
0/3: cnn
1/3: abcnews
2/3: cbsnews
Main title: abcnews
0/3: abcnews
1/3: breitbart
2/3: abcnews
Main title: abcnews
0/3: foxnews
1/3: cnn
2/3: cbsnews
Main title: abcnews
0/3: theguardian
1/3: breitbart
2/3: abcnews
Main title: abcnews
0/3: nbcnews
1/3: usatoday
2/3: usatoday
Main title: abcnews
0/3: abcnews
1/3: breitbart
2/3: usatoday
Main title: abcnews
0/3: usatoday
1/3: aol
2/3: usatoday
Main title: abcnews
0/3: breitbart
1/3: foxnews
2/3: usatoday
Main title: abcnews
0/3: foxnews
1/3: cbsnews
2/3: nbcnews
Main title: abcnews
0/3: abcnews
1/3: usatoday
2/3: nbcnews
Main title: abcnews
0/3: abcnews
1/3: nbcnews
2/3: usatoday
Main title: abcnews
0/3: cnbc
1/3: foxnews
2/3: cnbc
Main title: abcnews
0/3: aol
1/3: cnn
2/3: cnn
Main title: abcnews
0/3: foxnews
1/3: breitbart
2/3: cbsnews
Main title: abcnews
0/3: usatoday
1/3: cnn
2/3: abcnews
Main title: abcnews
0/3: infowars
1/3: breitbart
2/3: usatoday
Main ti

Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: usatoday
1/3: wsj
2/3: usatoday
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: cnn
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: breitbart
Main title: wsj
0/3: nbcnews
1/3: breitbart
2/3: breitbart
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: wsj
0/3: usatoday
1/3: wsj
2/3: wsj
Main title: wsj
0/3: wsj
1/3: usatoday
2/3: wsj
Main title: wsj
0/3: foxnews
1/3: usatoday
2/3: wsj
Main title: wsj
0/3: wsj
1/3: wsj
2/3: wsj
Main title: cnbc
0/3: usatoday
1/3: abcnews
2/3: reuters
Main title: cnbc
0/3: cnbc
1/3: cbsnews
2/3: abcnews
Main title: cnbc
0/3: cnbc
1/3: reuters
2/3: abcnews
Main title: 

Main title: usatoday
0/3: cnbc
1/3: usatoday
2/3: abcnews
Main title: usatoday
0/3: cnbc
1/3: cnbc
2/3: aol
Main title: usatoday
0/3: aol
1/3: aol
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: cnn
2/3: cnn
Main title: usatoday
0/3: usatoday
1/3: cnn
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: cnn
2/3: foxnews
Main title: usatoday
0/3: breitbart
1/3: usatoday
2/3: cbsnews
Main title: usatoday
0/3: usatoday
1/3: breitbart
2/3: breitbart
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: cnn
Main title: usatoday
0/3: usatoday
1/3: cnn
2/3: usatoday
Main title: usatoday
0/3: foxnews
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2

Main title: usatoday
0/3: foxnews
1/3: abcnews
2/3: aol
Main title: usatoday
0/3: cbsnews
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: aol
2/3: usatoday
Main title: usatoday
0/3: news9
1/3: nbcnews
2/3: breitbart
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: cnn
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: aol
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: abcnews
Main title: usatoday
0/3: breitbart
1/3: foxnews
2/3: abcnews
Main title: usatoday
0/3: cbsnews
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: usatoday
0/3: foxnews
1/3: usatoday
2/3: aol
Main title: usatoday
0/3: usatoday
1/3: cnn
2/3: usatoday
Main title: usatoday
0/3: aol
1/3: usatoday
2/3: foxnews
Main title: usatoday
0/3: foxnews
1/3: usatoday
2/3: cbsnews
Main title: usatoday
0/3: usatoday
1/3: cbsnews
2/3: cbsnews
Main title: usatoday
0/3: usatoday
1/3: us

Main title: cbsnews
0/3: wired
1/3: bbc
2/3: usatoday
Main title: cbsnews
0/3: breitbart
1/3: aol
2/3: cnn
Main title: cbsnews
0/3: aol
1/3: aol
2/3: usatoday
Main title: cbsnews
0/3: breitbart
1/3: cbsnews
2/3: foxnews
Main title: cbsnews
0/3: cbsnews
1/3: foxnews
2/3: cnn
Main title: cbsnews
0/3: usatoday
1/3: cnn
2/3: breitbart
Main title: cbsnews
0/3: abcnews
1/3: breitbart
2/3: abcnews
Main title: cbsnews
0/3: cbsnews
1/3: usatoday
2/3: reuters
Main title: cbsnews
0/3: usatoday
1/3: nbcnews
2/3: cbsnews
Main title: cbsnews
0/3: cbsnews
1/3: wired
2/3: cbsnews
Main title: cbsnews
0/3: cbsnews
1/3: abcnews
2/3: usatoday
Main title: cbsnews
0/3: cbsnews
1/3: cbsnews
2/3: cbsnews
Main title: cbsnews
0/3: cbsnews
1/3: wired
2/3: cbsnews
Main title: cbsnews
0/3: cbsnews
1/3: cbsnews
2/3: cbsnews
Main title: cbsnews
0/3: cnn
1/3: cbsnews
2/3: foxnews
Main title: cbsnews
0/3: usatoday
1/3: cbsnews
2/3: foxnews
Main title: cbsnews
0/3: cbsnews
1/3: foxnews
2/3: aol
Main title: cbsnews
0/3:

...

2/3: cnn
Main title: cnn
0/3: cbsnews
1/3: breitbart
2/3: cnn
Main title: cnn
0/3: cnn
1/3: cnn
2/3: usatoday
Main title: cnn
0/3: cnn
1/3: cnn
2/3: nbcnews
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cnn
1/3: cnn
2/3: abcnews
Main title: cnn
0/3: cnn
1/3: usatoday
2/3: cnn
Main title: cnn
0/3: nbcnews
1/3: cnn
2/3: cbsnews
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cnn
1/3: nbcnews
2/3: cbsnews
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: abcnews
1/3: usatoday
2/3: abcnews
Main title: cnn
0/3: cnn
1/3: cnn
2/3: usatoday
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: foxnews
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cnn
1/3: usatoday
2/3: cnn
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cnn
1/3: cnn
2/3: cnn
Main title: cnn
0/3: cbsnews
1/3: cbsnews
2/3: cnn
Main titl

1/3: cnn
2/3: theguardian
Main title: theguardian
0/3: foxnews
1/3: foxnews
2/3: aol
Main title: theguardian
0/3: cbsnews
1/3: cnn
2/3: wired
Main title: theguardian
0/3: cbsnews
1/3: theguardian
2/3: wired
Main title: theguardian
0/3: cnn
1/3: foxnews
2/3: cbsnews
Main title: theguardian
0/3: breitbart
1/3: cbsnews
2/3: usatoday
Main title: theguardian
0/3: breitbart
1/3: cbsnews
2/3: breitbart
Main title: theguardian
0/3: theguardian
1/3: washingtonpost
2/3: cnn
Main title: breitbart
0/3: infowars
1/3: abcnews
2/3: breitbart
Main title: breitbart
0/3: breitbart
1/3: infowars
2/3: infowars
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: breitbart
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: usatoday
Main title: breitbart
0/3: cbsnews
1/3: usatoday
2/3: washingtonpost
Main title: breitbart
0/3: foxnews
1/3: usatoday
2/3: usatoday
Main title: breitbart
0/3: breitbart
1/3: abcnews
2/3: bbc
Main title: breitbart
0/3: usatoday
1/3: foxnews
2/3: foxnews
Main title: brei

2/3: nbcnews
Main title: breitbart
0/3: usatoday
1/3: usatoday
2/3: news9
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: breitbart
Main title: breitbart
0/3: cnn
1/3: cnn
2/3: cnn
Main title: breitbart
0/3: breitbart
1/3: usatoday
2/3: cbsnews
Main title: breitbart
0/3: usatoday
1/3: usatoday
2/3: usatoday
Main title: breitbart
0/3: foxnews
1/3: breitbart
2/3: foxnews
Main title: breitbart
0/3: breitbart
1/3: usatoday
2/3: usatoday
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: cbsnews
Main title: breitbart
0/3: foxnews
1/3: cnbc
2/3: cnbc
Main title: breitbart
0/3: cnbc
1/3: foxnews
2/3: cnbc
Main title: breitbart
0/3: bbc
1/3: theguardian
2/3: aol
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: cbsnews
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: breitbart
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: foxnews
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: foxnews
Main title: breitbart
0/3: breitbart
1/3: breitbart
2/3: cbs

Main title: foxnews
0/3: wsj
1/3: abcnews
2/3: wsj
Main title: foxnews
0/3: abcnews
1/3: foxnews
2/3: abcnews
Main title: foxnews
0/3: breitbart
1/3: usatoday
2/3: foxnews
Main title: foxnews
0/3: usatoday
1/3: usatoday
2/3: cnn
Main title: foxnews
0/3: usatoday
1/3: usatoday
2/3: breitbart
Main title: foxnews
0/3: breitbart
1/3: usatoday
2/3: usatoday
Main title: foxnews
0/3: cbsnews
1/3: breitbart
2/3: usatoday
Main title: foxnews
0/3: breitbart
1/3: breitbart
2/3: usatoday
Main title: foxnews
0/3: foxnews
1/3: cbsnews
2/3: usatoday
Main title: foxnews
0/3: foxnews
1/3: foxnews
2/3: cbsnews
Main title: foxnews
0/3: breitbart
1/3: usatoday
2/3: breitbart
Main title: foxnews
0/3: foxnews
1/3: cnn
2/3: foxnews
Main title: foxnews
0/3: foxnews
1/3: foxnews
2/3: breitbart
Main title: foxnews
0/3: abcnews
1/3: abcnews
2/3: abcnews
Main title: foxnews
0/3: foxnews
1/3: usatoday
2/3: nbcnews
Main title: foxnews
0/3: foxnews
1/3: foxnews
2/3: foxnews
Main title: foxnews
0/3: foxnews
1/3: foxn

Main title: nbcnews
0/3: cnn
1/3: nbcnews
2/3: breitbart
Main title: nbcnews
0/3: cnn
1/3: nbcnews
2/3: cnn
Main title: nbcnews
0/3: nbcnews
1/3: nbcnews
2/3: cnn
Main title: nbcnews
0/3: nbcnews
1/3: breitbart
2/3: breitbart
Main title: nbcnews
0/3: abcnews
1/3: breitbart
2/3: breitbart
Main title: nbcnews
0/3: cbsnews
1/3: cbsnews
2/3: abcnews
Main title: nbcnews
0/3: abcnews
1/3: breitbart
2/3: cbsnews
Main title: nbcnews
0/3: breitbart
1/3: breitbart
2/3: wsj
Main title: nbcnews
0/3: nbcnews
1/3: nbcnews
2/3: nbcnews
Main title: nbcnews
0/3: breitbart
1/3: bbc
2/3: nbcnews
Main title: nbcnews
0/3: nbcnews
1/3: breitbart
2/3: nbcnews
Main title: nbcnews
0/3: breitbart
1/3: breitbart
2/3: nbcnews
Main title: nbcnews
0/3: breitbart
1/3: cnn
2/3: nbcnews
Main title: nbcnews
0/3: cnn
1/3: cnn
2/3: cnn
Main title: nbcnews
0/3: cnn
1/3: breitbart
2/3: nbcnews
Main title: nbcnews
0/3: nbcnews
1/3: nbcnews
2/3: nbcnews
Main title: nbcnews
0/3: nbcnews
1/3: nbcnews
2/3: nbcnews
Main title: n

In [7]:
print(avgArticleLen)

986.5774058577406


In [8]:
sentmatch = []
for i in range(n):
    mainarticle = wordRepArticles[i]
    rep=[]
    for j in range(matches):
        comparearticle = wordRepArticles[topmatches[i][j][1]]
        bestmatchSentence =[]
        for k in range(len(mainarticle[2])):
            val = -1
            msentence = mainarticle[2][k][1]
            mainlength = len(mainarticle[2][k][0].split(' '))
            for l in range(len(comparearticle[2])):
                csentence = comparearticle[2][l][1]
                comparelength = len(comparearticle[2][l][0].split(' '))
                commonwords = list(set(msentence.keys()) & set(csentence.keys())) 
                value = 0
                for word in commonwords:
                    mainvalue = (1+math.log(msentence[word]))*1.0/(math.log(1+mainlength/avgArticleLen))
                    compvalue = (1+math.log(csentence[word]))*1.0/(math.log(1+comparelength/avgArticleLen))
                    value += mainvalue*compvalue* math.log(1+n/idf[word])
                if value>val:
                    val = value
                    ind = l
            bestmatchSentence.append([ind,val,comparearticle[2][ind][0]])
        rep.append(bestmatchSentence)
    sentmatch.append(rep)

In [25]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() 

reliability = [9.9 for i in range(len(sources))]
articleaggreeement=[]
for i in range(n):
    articleval =0
    mainarticle = wordRepArticles[i]
    for j in range(matches):
        val=0
        comparticle=wordRepArticles[topmatches[i][j][1]]
        for k in range(len(mainarticle[2])):
#             print(sentmatch[i][j][k][2])
            sent1 = analyzer.polarity_scores(mainarticle[2][k][0])['compound']
            sent2 = analyzer.polarity_scores(sentmatch[i][j][k][2])['compound']
            val +=sent1*sent2
        val /= len(mainarticle[2])
        articleval +=val*reliability[sources.index(articles[topmatches[i][j][1]][0])]
    articleaggreeement.append(articleval/matches)

In [ ]:
import pickle
art = pickle.load(open('articles.pickle','rb'))